In [0]:
%sql
SELECT 
    customerID,
    CustomerFeedback,
    churn,
    ai_analyze_sentiment(CustomerFeedback) as sentiment_label,
    ai_classify(
        CustomerFeedback, 
        ARRAY('cost', 'quality', 'no_internet', 'Billing', 'competition', 'contract', 'personal_reason', 'relocation', 'customer_service', 'other')
    ) as categoria_reclamacao
FROM workspace.churn_zero.historico_bronze
WHERE sentiment_label == 'positive' AND churn = 'Yes'
LIMIT 50;